# Name: Uday Routhu

### Assginment: Applying Advanced Transformations (Core)

### Imports 

In [34]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [35]:
# Load the superhero_info.csv and superhero_powers.csv files into DataFrames
info_df = pd.read_csv('input/superhero_info - superhero_info.csv')
powers_df = pd.read_csv('input/superhero_powers - superhero_powers.csv')

In [36]:
info_df.info()
powers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [37]:
info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [38]:
powers_df.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [39]:
# Split the "Hero|Publisher" column into "Hero" and "Publisher" columns
info_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)

In [41]:
info_df.rename(columns={'Hero': 'hero_names'}, inplace=True)

In [42]:
# Merge the DataFrames on the 'Name' column to combine the information
merged_df = info_df.merge(powers_df, on='hero_names', how='left')

In [43]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
 8   hero_names      463 non-null    object
 9   Publisher       463 non-null    object
 10  Powers          463 non-null    object
dtypes: object(11)
memory usage: 43.4+ KB


In [44]:
merged_df.drop(columns=['Publisher'], inplace=True)

In [45]:
# Split the "Hero|Publisher" column into "Hero" and "Publisher" columns
merged_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)

In [46]:
merged_df.drop(columns=['Hero|Publisher'], inplace=True)

In [49]:
# Use str.extract to extract Height and Weight as numeric values
merged_df[['Height (numeric)', 'Weight (numeric)']] = merged_df['Measurements'].str.extract(r"'Height': '(?P<Height_numeric>\d+\.\d+) cm', 'Weight': '(?P<Weight_numeric>\d+\.\d+) kg'")

In [50]:
# Convert the extracted columns to numeric
merged_df['Height (numeric)'] = pd.to_numeric(merged_df['Height (numeric)'])
merged_df['Weight (numeric)'] = pd.to_numeric(merged_df['Weight (numeric)'])

In [63]:
merged_df.drop(columns=['Measurements'], inplace=True)
merged_df.drop(columns=['Powers'], inplace=True)

In [72]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            463 non-null    object 
 1   Race              463 non-null    object 
 2   Alignment         463 non-null    object 
 3   Hair color        463 non-null    object 
 4   Eye color         463 non-null    object 
 5   Skin color        463 non-null    object 
 6   hero_names        463 non-null    object 
 7   Hero              463 non-null    object 
 8   Publisher         463 non-null    object 
 9   Height (numeric)  463 non-null    float64
 10  Weight (numeric)  463 non-null    float64
dtypes: float64(2), object(9)
memory usage: 59.6+ KB


In [65]:
# Split the comma-separated powers into lists
powers_df['Powers'] = powers_df['Powers'].str.split(',')

In [66]:
# One-hot-encode the powers
encoded_powers  = powers_df['Powers'].str.join('|').str.get_dummies()

In [68]:
# Concatenate the one-hot-encoded powers with the original DataFrame
result_df = pd.concat([merged_df, encoded_powers], axis=1)

In [70]:
print(result_df.columns.tolist())

['Gender', 'Race', 'Alignment', 'Hair color', 'Eye color', 'Skin color', 'hero_names', 'Hero', 'Publisher', 'Height (numeric)', 'Weight (numeric)', 'Accelerated Healing', 'Adaptation', 'Agility', 'Animal Attributes', 'Animal Control', 'Animal Oriented Powers', 'Animation', 'Anti-Gravity', 'Astral Projection', 'Astral Travel', 'Audio Control', 'Banish', 'Biokinesis', 'Camouflage', 'Changing Armor', 'Clairvoyance', 'Cloaking', 'Cold Resistance', 'Cryokinesis', 'Danger Sense', 'Darkforce Manipulation', 'Death Touch', 'Density Control', 'Dexterity', 'Dimensional Awareness', 'Dimensional Travel', 'Duplication', 'Durability', 'Echolocation', 'Elasticity', 'Electrical Transport', 'Electrokinesis', 'Element Control', 'Elemental Transmogrification', 'Empathy', 'Energy Absorption', 'Energy Armor', 'Energy Beams', 'Energy Blasts', 'Energy Constructs', 'Energy Manipulation', 'Energy Resistance', 'Enhanced Hearing', 'Enhanced Memory', 'Enhanced Senses', 'Enhanced Sight', 'Enhanced Smell', 'Enhanced